# Demohuset into Azure Digital Twins 
## An example of the Digital Twin

This example just includes:
* Connecting to an ADT environment
* Uploading models (using Azure Digital Twins Explorer )
* Creating twins og building, level and rooms based on IFC model
* Querying twins

### After completing this tutorial part, you will have gone from ARK.ifc to Buildings, its levels and ints rooms in Azure Digital Twins
![IFC viewer to Azure Digital Twins view](img/ifc_to_rec.png "IFC viewer to Azure Digital Twins view")

Note: We are using IfcOpenShell to parse the provided IFC models. You find the [dokumentation here](https://docs.ifcopenshell.org/ifcopenshell-python/hello_world.html)

[This is the SDK repo on Github](https://github.com/Azure/azure-sdk-for-python/tree/4559e19e2f3146a49f1eba1706bb798071f4a1f5/sdk/digitaltwins/azure-digitaltwins-core)

[Here is the doc on the query language](https://docs.microsoft.com/en-us/azure/digital-twins/concepts-query-language)


## Connecting

* **note 1**: you will need to replace {`demohuset.api.weu.digitaltwins.azure.net`} with the address of your ADT. 
* **note 2**: you will need to replace your tenant id as well 

In [2]:
# Need to login to the environment for 

#! az login --tenant "{paste your tenant id here}"

In [38]:
from azure.identity import AzureCliCredential
from azure.digitaltwins.core import DigitalTwinsClient

your_digital_twin_url = "demohuset.api.weu.digitaltwins.azure.net" 

azure_cli = AzureCliCredential()
service_client = DigitalTwinsClient(
    your_digital_twin_url, azure_cli)
service_client

# REC/Brick Models in ADT
This tutorial assumes that you have uploaded the model found in [this repository](https://github.com/mok-see/rec/tree/university_dev)

It is a specialized model developed based on [REC/BrickSchema](https://dev.realestatecore.io/)

Open Azure Digtial Twins Explorer and upload models as showed below.

![Azure Digital Twin Explorer UI](img/adt_explorer.png "Azure Digital Twin Explorer")



## Creating twins

The twin data is based on Demohuset ifc files as found in "/demohuset" folder

The ARK model looks like this

![IFC Model in Viewer](img/openifcviewer.png "OpenIFCViewer")

We will use ifcopenshell to parse out information on Building, Storeys and rooms in this model. 
[Learn more about the library here](https://docs.ifcopenshell.org/ifcopenshell-python/hello_world.html)



In [72]:
# Use ifcopenshell to parse the models. We will start with the Architecture model
import ifcopenshell

ark = ifcopenshell.open("demohuset/ARK.ifc")

# Lets grab the Building, Storeys and rooms from that model 

ifc_buildings = ark.by_type("IfcBuilding")
ifc_storeys = ark.by_type("IfcBuildingStorey")
ifc_spaces = ark.by_type("IfcSpace")

### Common helper functions 

In [73]:
# Need to import our REC models 
from rec_models import Architecture, Building, Level, Room

def upsert_ifc_twin(twin: Building | Level | Room ):
    print(twin.model_dump(exclude_none=True,by_alias=True))
    service_client.upsert_digital_twin(twin.dtid, twin.model_dump(exclude_none=True,by_alias=True))

# Function to create and upsert the relationship between a building->levels and levels -> rooms
def add_part_relationship(source : Architecture, target : Architecture):
    # hasPart
    relationship_json = {
        "$relationshipId": f"{source.dtid}-hasPart-{target.dtid}",
        "$sourceId": source.dtid,
        "$relationshipName": "hasPart",
        "$targetId": target.dtid,
    }
    service_client.upsert_relationship(
        relationship_json["$sourceId"],
        relationship_json["$relationshipId"],
        relationship_json,
    )
    # isPartOf
    relationship_json = {
        "$relationshipId": f"{target.dtid}-isPartOf-{source.dtid}",
        "$sourceId": target.dtid,
        "$relationshipName": "isPartOf",
        "$targetId": source.dtid,
    }
    service_client.upsert_relationship(
        relationship_json["$sourceId"],
        relationship_json["$relationshipId"],
        relationship_json,
    )

def get_twin_by_ifc_guid(ifc_guid):
    query_expression = "SELECT * FROM digitaltwins t WHERE t.identifiers.IfcGuid = '{}'".format(
        ifc_guid
    )
    query_result = service_client.query_twins(query_expression)
    values = []
    for i in query_result:
        values.append(i)
    if len(values) > 1: 
        ## exception -- can only be one global unique IFC guid
        raise Exception("There is more than one twin with this guid, can only be one global unique IFC guid")
    else:
        return values[0]
        

### Let us create and upsert some twins 


#### Building 
* [DTMI: dtmi:org:w3id:rec:Building;1](https://dev.realestatecore.io/ontology/Space/Architecture/Building/Building)
* [IfcBuilding docs](https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifcproductextension/lexical/ifcbuilding.htm)

In [74]:
for building in ifc_buildings:
    building_twin = Building(name=building.Name,identifiers={"IfcGuid":building.GlobalId})
    upsert_ifc_twin(building_twin)
    

{'$dtId': 'Building-2bde74d5-a99f-4a5b-add3-0c9032191932', 'name': 'Asplund', 'identifiers': {'IfcGuid': '3oUjp0LRHDfvVS2UDNWsfz'}, 'customProperties': {}, '$metadata': {'$model': 'dtmi:org:w3id:rec:Building;1'}, 'area': {'name': '', 'customProperties': {}, 'customTags': {}, '$metadata': {}}, 'capacity': {'name': '', 'customProperties': {}, 'customTags': {}, '$metadata': {}}}


#### In the Azure Digital Twin it should look like this after "SELECT * FROM digitaltwins"

![ADT Explorer Building Twin](img/building_twin.png "ADT Explorer Building Twin")

#### Building Storeys aka Levels  
* [DTMI: dtmi:org:w3id:rec:Level;1](https://dev.realestatecore.io/ontology/Space/Architecture/Level/Level)
* [IfcBuildingStorey docs](https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifcproductextension/lexical/ifcbuildingstorey.htm)

In [75]:
for storey in ifc_storeys:
    level_twin = Level(name=storey.Name,identifiers={"IfcGuid":storey.GlobalId})
    upsert_ifc_twin(level_twin)

{'$dtId': 'Level-0e476f26-0971-4abf-bb65-fac0b87d5b1a', 'name': '-2. Fundamentplan', 'identifiers': {'IfcGuid': '02L27m6dDFoAtcd48OiKG5'}, 'customProperties': {}, '$metadata': {'$model': 'dtmi:org:w3id:rec:Level;1'}, 'area': {'name': '', 'customProperties': {}, 'customTags': {}, '$metadata': {}}, 'capacity': {'name': '', 'customProperties': {}, 'customTags': {}, '$metadata': {}}}
{'$dtId': 'Level-c5877f84-00ca-42aa-b161-fa9e79a82dea', 'name': '-1. Underetasje', 'identifiers': {'IfcGuid': '18DVBjQGbDNh4nL4pLc3T7'}, 'customProperties': {}, '$metadata': {'$model': 'dtmi:org:w3id:rec:Level;1'}, 'area': {'name': '', 'customProperties': {}, 'customTags': {}, '$metadata': {}}, 'capacity': {'name': '', 'customProperties': {}, 'customTags': {}, '$metadata': {}}}
{'$dtId': 'Level-bb88455d-9cb8-4be2-8d24-4efc0dad10ef', 'name': '0. 1. etasje', 'identifiers': {'IfcGuid': '3swGzdAGvB7OfkTWz$ss3m'}, 'customProperties': {}, '$metadata': {'$model': 'dtmi:org:w3id:rec:Level;1'}, 'area': {'name': '', 'cu

#### In the Azure Digital Twin it should now look like this after "SELECT * FROM digitaltwins"

![ADT Explorer Building and Level twins](img/building_and_level_twins.png "ADT Explorer Building and Level twins")

#### Spaces aka Rooms
* [DTMI: dtmi:org:w3id:rec:Room;1](https://dev.realestatecore.io/ontology/Space/Architecture/Room/Room)
* [IfcSpace docs](https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifcproductextension/lexical/ifcspace.htm)

In [ ]:
import ifcopenshell.util.element as api
for space in ifc_spaces:
    psets = api.get_psets(space)
    if 'BaseQuantities' in psets and "NetFloorArea" in psets['BaseQuantities'] and "GrossFloorArea" in psets['BaseQuantities']:
        bq_areas = {"netArea": psets['BaseQuantities']["NetFloorArea"], "grossArea": psets['BaseQuantities']["GrossFloorArea"]}
    else:
        bq_areas = {}
    room_twin = Room(name=space.Name,identifiers={"IfcGuid":space.GlobalId},area=bq_areas)
    upsert_ifc_twin(room_twin)

{'$dtId': 'Room-61ce1897-44c1-4499-b924-86539e4305ae', 'name': '07', 'identifiers': {'IfcGuid': '3nEpfmJzn4lfik7wGpJyAt'}, 'customProperties': {}, '$metadata': {'$model': 'dtmi:org:w3id:rec:Room;1'}, 'area': {'name': '', 'customProperties': {}, 'customTags': {}, '$metadata': {}, 'grossArea': 23.814728, 'netArea': 23.814728}, 'capacity': {'name': '', 'customProperties': {}, 'customTags': {}, '$metadata': {}}}
{'$dtId': 'Room-4c9411a0-93a6-48d7-92f1-c7fe8bb8fc2b', 'name': '08', 'identifiers': {'IfcGuid': '0KmPi6DIb3OOj1yzsHQeAC'}, 'customProperties': {}, '$metadata': {'$model': 'dtmi:org:w3id:rec:Room;1'}, 'area': {'name': '', 'customProperties': {}, 'customTags': {}, '$metadata': {}, 'grossArea': 10.938164, 'netArea': 10.938164}, 'capacity': {'name': '', 'customProperties': {}, 'customTags': {}, '$metadata': {}}}
{'$dtId': 'Room-6ab6333a-367b-425c-93c6-33af6dc8c8e2', 'name': '07', 'identifiers': {'IfcGuid': '0q24Bo_G940wCmX0qqAnot'}, 'customProperties': {}, '$metadata': {'$model': 'dtmi

#### In the Azure Digital Twin it should now look like this after "SELECT * FROM digitaltwins"

![ADT Explorer Building, Level and room twins](img/building_levels_and_room_twins.png "ADT Explorer Building, Level and room twins")

#### Then we need to make the connections in the graph

The topology of the building can be represented using part relationships. hasPart and the inverse, isPartOf, as listed on eg. [REC documentation for room here.](https://dev.realestatecore.io/ontology/Space/Architecture/Room/Room#inherited-relationships)

How can we find which to relate betwen? Levels and Buildings are easy as we only have one Building, but how about Rooms? Well, these relationships are present in the IFC model. It is called [Spatial Decomposition](https://standards.buildingsmart.org/IFC/DEV/IFC4_2/FINAL/HTML/schema/ifcproductextension/lexical/ifcbuilding.htm#:~:text=IfcBuilding%20has%20PARTIAL) that can be accessed through the IsDecomposedBy property that holds a [IfcRelAggregate](https://iaiweb.lbl.gov/Resources/IFC_Releases/R2x3_final/ifckernel/lexical/ifcrelaggregates.htm) that relates the (in this case) Building with its floors and Storeys with its rooms. 

In [77]:
## Relating the building to its levels
for building in ifc_buildings:

    building_twin = Building(**get_twin_by_ifc_guid(building.GlobalId))
    print(building_twin)
    for rel_aggregate in building.IsDecomposedBy:
        related_storeys = rel_aggregate.RelatedObjects 
        for storey in related_storeys:
            storey_twin = Level(**get_twin_by_ifc_guid(storey.GlobalId))
            print("Relating: \n\t", storey_twin)
            add_part_relationship(building_twin,storey_twin)
            


dtid='Building-2bde74d5-a99f-4a5b-add3-0c9032191932' name='Asplund' identifiers={'IfcGuid': '3oUjp0LRHDfvVS2UDNWsfz'} customProperties={} metadata=BuildingMetadata(model='dtmi:org:w3id:rec:Building;1', name=LastUpdateTime(lastUpdateTime=datetime.datetime(2025, 1, 23, 5, 39, 21, 125926, tzinfo=TzInfo(UTC))), identifiers=LastUpdateTime(lastUpdateTime=datetime.datetime(2025, 1, 23, 5, 39, 21, 125926, tzinfo=TzInfo(UTC))), customProperties=LastUpdateTime(lastUpdateTime=datetime.datetime(2025, 1, 23, 5, 39, 21, 125926, tzinfo=TzInfo(UTC))), lastUpdateTime=datetime.datetime(2025, 1, 23, 5, 39, 21, 125926, tzinfo=TzInfo(UTC))) area=ArchitectureArea(dtid=None, name='', identifiers=None, customProperties={}, customTags={}, metadata=ArchitectureAreaMetadata(model=None, name=LastUpdateTime(lastUpdateTime=datetime.datetime(2025, 1, 23, 5, 39, 21, 125926, tzinfo=TzInfo(UTC))), identifiers=None, customProperties=LastUpdateTime(lastUpdateTime=datetime.datetime(2025, 1, 23, 5, 39, 21, 125926, tzinfo=T

#### Now you should see some related twins and some roming rooms when executing "SELECT * FROM digitaltwins"

![ADT Explorer Building related to Levels](img/building_related_to_levels.png "ADT Explorer Building related to Levels")

In [80]:
# Relating the rooms to its levels 
for storey in ifc_storeys:
    storey_twin = Level(**get_twin_by_ifc_guid(storey.GlobalId))
    print(storey_twin)
    for rel_aggregate in storey.IsDecomposedBy:
        related_spaces = rel_aggregate.RelatedObjects 
        for space in related_spaces:
            room_twin = Room(**get_twin_by_ifc_guid(space.GlobalId))
            print("Relating: \n\t", room_twin)
            add_part_relationship(storey_twin,room_twin)
            

dtid='Level-0e476f26-0971-4abf-bb65-fac0b87d5b1a' name='-2. Fundamentplan' identifiers={'IfcGuid': '02L27m6dDFoAtcd48OiKG5'} customProperties={} metadata=LevelMetadata(model='dtmi:org:w3id:rec:Level;1', name=LastUpdateTime(lastUpdateTime=datetime.datetime(2025, 1, 23, 5, 39, 25, 348336, tzinfo=TzInfo(UTC))), identifiers=LastUpdateTime(lastUpdateTime=datetime.datetime(2025, 1, 23, 5, 39, 25, 348336, tzinfo=TzInfo(UTC))), customProperties=LastUpdateTime(lastUpdateTime=datetime.datetime(2025, 1, 23, 5, 39, 25, 348336, tzinfo=TzInfo(UTC))), lastUpdateTime=datetime.datetime(2025, 1, 23, 5, 39, 25, 348336, tzinfo=TzInfo(UTC))) area=ArchitectureArea(dtid=None, name='', identifiers=None, customProperties={}, customTags={}, metadata=ArchitectureAreaMetadata(model=None, name=LastUpdateTime(lastUpdateTime=datetime.datetime(2025, 1, 23, 5, 39, 25, 348336, tzinfo=TzInfo(UTC))), identifiers=None, customProperties=LastUpdateTime(lastUpdateTime=datetime.datetime(2025, 1, 23, 5, 39, 25, 348336, tzinfo=

#### Now it looks like this after a bit rearranging after running "SELECT * FROM digitaltwins"

![ADT Explorer Building related to Levels and its rooms](img/connected_graph.png "ADT Explorer Building related to Levels and its rooms")

As the image shows, the rooms are only on two levels. 

Now, how about connecting the rest of the models to the same twin environment and maybe start connecting other data sources? 